<a href="https://colab.research.google.com/github/juanjose1919/Proyectos_tercer_corte/blob/main/Proyecto_10_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install squarify
import statistics
import squarify as sq
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import pandas as pd
import pickle as pkl
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.datasets import make_regression, make_classification
import statistics as sta
import warnings
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
warnings.filterwarnings("ignore")
train=pd.read_csv("/content/train.csv")
test=pd.read_csv("/content/test.csv")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Exploración inicial

In [ ]:
train.head()

,target,id,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,0,894,51.0,1.0,0.0,140.0,299.0,0.0,1.0,173.0,1.0,1.6,2.0,0.0,3.0
1,1,204,66.0,0.0,2.0,146.0,278.0,0.0,0.0,152.0,0.0,0.0,1.0,1.0,2.0
2,1,575,43.0,1.0,0.0,150.0,247.0,0.0,1.0,171.0,NaN,1.5,2.0,0.0,2.0
3,0,621,48.0,1.0,0.0,130.0,256.0,1.0,NaN,150.0,1.0,0.0,2.0,2.0,3.0
4,0,660,61.0,1.0,0.0,138.0,166.0,0.0,0.0,125.0,1.0,3.6,1.0,1.0,2.0


In [44]:
test.tail()

,id,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
252,757,66.0,1.0,0.0,160.0,228.0,0.0,0.0,138.0,0.0,2.3,2.0,0.0,1.0
253,765,57.0,1.0,0.0,110.0,335.0,0.0,1.0,143.0,1.0,3.0,1.0,1.0,3.0
254,754,51.0,0.0,2.0,130.0,256.0,0.0,0.0,149.0,0.0,0.5,2.0,0.0,2.0
255,191,56.0,1.0,1.0,130.0,221.0,0.0,0.0,163.0,0.0,0.0,2.0,0.0,3.0
256,38,64.0,1.0,0.0,128.0,263.0,0.0,1.0,105.0,1.0,0.2,1.0,1.0,3.0


In [ ]:
train.duplicated().sum()

0

In [ ]:
test.duplicated().sum()

0

In [ ]:
train.isna().sum()

target       0
id           0
age         46
sex         47
cp          46
trestbps    48
chol        44
fbs         47
restecg     43
thalach     38
exang       37
oldpeak     42
slope       38
ca          31
thal        37
dtype: int64

In [ ]:
test.isna().sum()

id           0
age          1
sex          1
cp           2
trestbps     1
chol         4
fbs          2
restecg      5
thalach     10
exang       11
oldpeak      5
slope       12
ca          17
thal         9
dtype: int64

In [ ]:
train.nunique()

target        2
id          768
age          41
sex           2
cp            4
trestbps     49
chol        149
fbs           2
restecg       3
thalach      90
exang         2
oldpeak      39
slope         3
ca            5
thal          4
dtype: int64

### Tratamiento de missing values

In [ ]:
test["trestbps"].mean()

131.21484375

In [ ]:
filltrain=train.copy()

### Para limpiar el DataFrame se obtiene la moda de las variables categoricas

In [ ]:
filltrain['cp']=train["cp"].fillna(statistics.mode(train["cp"]))
filltrain['fbs']=train["fbs"].fillna(statistics.mode(train["fbs"]))
filltrain['restecg']=train["restecg"].fillna(statistics.mode(train["restecg"]))
filltrain['exang']=train["exang"].fillna(statistics.mode(train["exang"]))
filltrain['slope']=train["slope"].fillna(statistics.mode(train["slope"]))
filltrain['ca']=train["ca"].fillna(statistics.mode(train["ca"]))
filltrain['thal']=train["thal"].fillna(statistics.mode(train["thal"]))

In [ ]:
filltrain.isna().sum()

target       0
id           0
age         46
sex         47
cp           0
trestbps    48
chol        44
fbs          0
restecg      0
thalach     38
exang        0
oldpeak     42
slope        0
ca           0
thal         0
dtype: int64

### Ahora predeciremos los datos faltantes con las variables que previamente se les inyectó la moda 

In [ ]:
age_not_null = filltrain.drop(filltrain[filltrain.age.isnull()].index)
age_null = filltrain.drop(age_not_null.index)
features = ['cp','fbs','restecg','exang', 'slope', 'ca', 'thal']
y = age_not_null['age']
x = age_not_null[features]
model = LinearRegression()
model.fit(x,y)
x = age_null[features]
predicted_age = model.predict(x)
filltrain.loc[filltrain.age.isnull(), "age"] = predicted_age
filltrain.age.isna().sum()

0

In [ ]:
sex_not_null = filltrain.drop(filltrain[filltrain.sex.isnull()].index)
sex_null = filltrain.drop(sex_not_null.index)
features = ['cp','fbs','restecg','exang', 'slope', 'ca', 'thal']
y = sex_not_null['sex']
x = sex_not_null[features]
model = LinearRegression()
model.fit(x,y)
x = sex_null[features]
predicted_sex = model.predict(x)
filltrain.loc[filltrain.sex.isnull(), "sex"] = predicted_sex
filltrain.sex.isna().sum()

0

In [ ]:
trestbps_not_null = filltrain.drop(filltrain[filltrain.trestbps.isnull()].index)
trestbps_null = filltrain.drop(trestbps_not_null.index)
features = ['cp','fbs','restecg','exang', 'slope', 'ca', 'thal']
y = trestbps_not_null['trestbps']
x = trestbps_not_null[features]
model = LinearRegression()
model.fit(x,y)
x = trestbps_null[features]
predicted_trestbps = model.predict(x)
filltrain.loc[filltrain.trestbps.isnull(), "trestbps"] = predicted_trestbps
filltrain.trestbps.isna().sum()

0

In [ ]:
chol_not_null = filltrain.drop(filltrain[filltrain.chol.isnull()].index)
chol_null = filltrain.drop(chol_not_null.index)
features = ['cp','fbs','restecg','exang', 'slope', 'ca', 'thal']
y = chol_not_null['chol']
x = chol_not_null[features]
model = LinearRegression()
model.fit(x,y)
x = chol_null[features]
predicted_chol = model.predict(x)
filltrain.loc[filltrain.chol.isnull(), "chol"] = predicted_chol
filltrain.chol.isna().sum()

0

In [ ]:
thalach_not_null = filltrain.drop(filltrain[filltrain.thalach.isnull()].index)
thalach_null = filltrain.drop(thalach_not_null.index)
features = ['cp','fbs','restecg','exang', 'slope', 'ca', 'thal']
y = thalach_not_null['thalach']
x = thalach_not_null[features]
model = LinearRegression()
model.fit(x,y)
x = thalach_null[features]
predicted_thalach = model.predict(x)
filltrain.loc[filltrain.thalach.isnull(), "thalach"] = predicted_thalach
filltrain.thalach.isna().sum()

0

In [ ]:
oldpeak_not_null = filltrain.drop(filltrain[filltrain.oldpeak.isnull()].index)
oldpeak_null = filltrain.drop(oldpeak_not_null.index)
features = ['cp','fbs','restecg','exang', 'slope', 'ca', 'thal']
y = oldpeak_not_null['oldpeak']
x = oldpeak_not_null[features]
model = LinearRegression()
model.fit(x,y)
x = oldpeak_null[features]
predicted_oldpeak = model.predict(x)
filltrain.loc[filltrain.oldpeak.isnull(), "oldpeak"] = predicted_oldpeak
filltrain.oldpeak.isna().sum()

0

In [ ]:
filltrain.isna().sum()

target      0
id          0
age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
dtype: int64

In [ ]:
filltest=test.copy()

filltest['trestbps']=test["trestbps"].fillna(test["trestbps"].mean())
filltest['chol']=test["chol"].fillna(test["chol"].mean())
filltest['thalach']=test["thalach"].fillna(test["thalach"].mean())
filltest['oldpeak']=test["oldpeak"].fillna(test["oldpeak"].mean())
filltest['age']=test["age"].fillna(test["age"].mean())


filltest['sex']=test["sex"].fillna(statistics.mode(test["sex"]))
filltest['cp']=test["cp"].fillna(statistics.mode(test["cp"]))
filltest['fbs']=test["fbs"].fillna(statistics.mode(test["fbs"]))
filltest['restecg']=test["restecg"].fillna(statistics.mode(test["restecg"]))
filltest['exang']=test["exang"].fillna(statistics.mode(test["exang"]))
filltest['slope']=test["slope"].fillna(statistics.mode(test["slope"]))
filltest['ca']=test["ca"].fillna(statistics.mode(test["ca"]))
filltest['thal']=test["thal"].fillna(statistics.mode(test["thal"]))

In [ ]:
filltrain = filltrain.drop(columns=['id'])

In [ ]:
filltest = filltest.drop(columns=['id'])

### modelo

In [ ]:
y_train=filltrain[['target']]
X_train=filltrain.drop(columns=['target'])

In [ ]:
rf_cls = RandomForestClassifier(n_estimators=100)
rf_cls.fit(X_train,y_train)

RandomForestClassifier()

In [ ]:
y_fit_train = rf_cls.predict(X_train)
y_pred = rf_cls.predict(filltest)

In [ ]:
y_pred

array([0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1])

In [ ]:
def metricas_clas(y_true, y_pred):
  print(f'Accuracy: {accuracy_score(y_true, y_pred)}')
  print(f"Precision: {precision_score(y_true, y_pred, average='weighted')}")
  print(f"Recall: {recall_score(y_true, y_pred,average='weighted')}")

In [ ]:
print('---'*10)
print('Métricas en entrenamiento')
print('---'*10)
metricas_clas(y_train, y_fit_train)
print('---'*10)

------------------------------
Métricas en entrenamiento
------------------------------
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
------------------------------


In [ ]:
metricas_clas(y_train, y_fit_train)

Accuracy: 1.0
Precision: 1.0
Recall: 1.0


In [ ]:
id=test['id']

In [ ]:
resultados = pd.DataFrame({'id':id,'Predicted':y_pred})
resultados

,id,Predicted
0,822,0
1,673,1
2,262,1
3,568,1
4,483,1
...,...,...
252,757,1
253,765,0
254,754,1
255,191,1


In [ ]:
resultados.to_csv('resultados.csv',index=False)